In [1]:
import argparse
import torch
import numpy as np
import os
import datetime
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import time

from collections import OrderedDict
from typing import List
import flwr as fl
import matplotlib.pyplot as plt

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

DEVICE = torch.device("cpu")
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 0.18.0
numpy 1.21.5
torch 1.11.0+cu102
torchvision 0.12.0+cu102
Training on cpu


## Data Loading and Preprocessing

In [2]:
import pandas as pd
df = pd.read_csv('./jd_computer_final1.csv')

In [3]:
len(df)

1429297

In [4]:
NUM_CLIENTS = 299

In [5]:
session = []
session = df['Session_ID'].unique()

In [6]:
session

array([     0,      1,      6, ..., 999995, 999997, 999999])

In [7]:
partition_size = 999999 // NUM_CLIENTS

In [8]:
len(session)

488656

In [9]:
partition_size

3344

In [10]:
df2 = [0 for i in range(NUM_CLIENTS + 1)]

In [11]:
for i in range(NUM_CLIENTS + 1):
    if i == 0:
        df2[i] = df[df['Session_ID'].between((i * partition_size) - 1, ((i+1) * partition_size), inclusive='right')]
    else:
        df2[i] = df[df['Session_ID'].between((i * partition_size), ((i+1) * partition_size), inclusive='right')]
    

In [12]:
len(df2)

300

In [13]:
len_val = partition_size // 10 

In [14]:
len_val

334

In [15]:
trainclient = [0 for i in range(NUM_CLIENTS + 1)]
valclient = [0 for i in range(NUM_CLIENTS + 1)]
for i in range(NUM_CLIENTS + 1):
    trainclient[i] = df2[i][df2[i]['Session_ID'] < (((i+1) * partition_size) - len_val)]
    valclient[i] = df2[i][df2[i]['Session_ID'] >= (((i+1) * partition_size) - len_val)]

In [16]:
class Dataset(object):
    def __init__(self, path, sep=',', session_key='Session_ID', item_key='category', time_key='Seq', n_sample=-1, itemmap=None, itemstamp=None, time_sort=True):
        # Read csv
        #self.df = pd.read_csv(path, sep=sep, dtype={session_key: int, item_key: int, time_key: float})
        self.df = path
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        if n_sample > 0:
            self.df = self.df[:n_sample]

        # Add colummn item index to data
        self.add_item_indices(itemmap=itemmap)
        """
        Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        clicks within a session are next to each other, where the clicks within a session are time-ordered.
        """
        self.df.sort_values([session_key, time_key], inplace=True)
        self.click_offsets = self.get_click_offset()
        self.session_idx_arr = self.order_session_idx()

    def add_item_indices(self, itemmap=None):
        """
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # type is numpy.ndarray
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            # Build itemmap is a DataFrame that have 2 columns (self.item_key, 'item_idx)
            itemmap = pd.DataFrame({self.item_key: item_ids,
                                   'item_idx': item2idx[item_ids].values})
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')

    def get_click_offset(self):
        """
        self.df[self.session_key] return a set of session_key
        self.df[self.session_key].nunique() return the size of session_key set (int)
        self.df.groupby(self.session_key).size() return the size of each session_id
        self.df.groupby(self.session_key).size().cumsum() retunn cumulative sum
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()
        return offsets

    def order_session_idx(self):
        if self.time_sort:
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())
        return session_idx_arr

    @property
    def items(self):
        return self.itemmap[self.item_key].unique()

In [17]:
class DataLoader():
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.

        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.

        Yields:
            input (B,): torch.FloatTensor. Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """
        # initializations
        df = self.dataset.df
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = []  # indicator for the sessions to be terminated
        finished = False

        while not finished:
            minlen = (end - start).min()
            # Item indices(for embedding) for clicks where the first sessions start
            idx_target = df.item_idx.values[start]

            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                input = torch.LongTensor(idx_input)
                target = torch.LongTensor(idx_target)
                yield input, target, mask

            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many sessions should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

In [18]:
train_data = [0 for i in range(NUM_CLIENTS + 1)]
valid_data = [0 for i in range(NUM_CLIENTS + 1)]
for i in range(NUM_CLIENTS + 1):
    train_data[i] = Dataset(trainclient[i])
    valid_data[i] = Dataset(valclient[i])

## Single Setting

In [19]:
class TOP1_max(nn.Module):
    def __init__(self):
        super(TOP1_max, self).__init__()

    def forward(self, logit):
        logit_softmax = F.softmax(logit, dim=1)
        diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)
        loss = torch.mean(logit_softmax * (torch.sigmoid(diff) + torch.sigmoid(logit ** 2)))
        return loss
    
class TOP1Loss(nn.Module):
    def __init__(self):
        super(TOP1Loss, self).__init__()
    def forward(self, logit):
        """
        Args:
            logit (BxB): Variable that stores the logits for the items in the mini-batch
                         The first dimension corresponds to the batches, and the second
                         dimension corresponds to sampled number of items to evaluate
        """
        diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)
        loss = torch.sigmoid(diff).mean() + torch.sigmoid(logit ** 2).mean()
        return loss    
    
class LossFunction(nn.Module):
    def __init__(self, loss_type='TOP1', use_cuda=False):
        """ An abstract loss function that can supports custom loss functions compatible with PyTorch."""
        super(LossFunction, self).__init__()
        self.loss_type = loss_type
        self.use_cuda = use_cuda
        if loss_type == 'TOP1-max':
            self._loss_fn = TOP1_max()
        elif loss_type == 'TOP1':
            self._loss_fn = TOP1Loss()
        else:
            raise NotImplementedError

    def forward(self, logit):
        return self._loss_fn(logit) 

In [20]:
loss_function = LossFunction('TOP1-max')

In [21]:
class GRU4REC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, final_act='tanh',
                 dropout_hidden=.5, dropout_input=0, batch_size=50, embedding_dim=-1, use_cuda=False):
        super(GRU4REC, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout_hidden = dropout_hidden
        self.dropout_input = dropout_input
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.use_cuda = use_cuda
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        self.onehot_buffer = self.init_emb()
        self.h2o = nn.Linear(hidden_size, output_size)
        self.create_final_activation(final_act)
        if self.embedding_dim != -1:
            self.look_up = nn.Embedding(input_size, self.embedding_dim)
            self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.num_layers, dropout=self.dropout_hidden)
        else:
            self.gru = nn.GRU(self.input_size, self.hidden_size, self.num_layers, dropout=self.dropout_hidden)
        self = self.to(self.device)

    def create_final_activation(self, final_act):
        if final_act == 'tanh':
            self.final_activation = nn.Tanh()
        elif final_act == 'relu':
            self.final_activation = nn.ReLU()
        elif final_act == 'softmax':
            self.final_activation = nn.Softmax()
        elif final_act == 'softmax_logit':
            self.final_activation = nn.LogSoftmax()
        elif final_act.startswith('elu-'):
            self.final_activation = nn.ELU(alpha=float(final_act.split('-')[1]))
        elif final_act.startswith('leaky-'):
            self.final_activation = nn.LeakyReLU(negative_slope=float(final_act.split('-')[1]))

    def forward(self, input, hidden):
        '''
        Args:
            input (B,): a batch of item indices from a session-parallel mini-batch.
            target (B,): torch.LongTensor of next item indices from a session-parallel mini-batch.

        Returns:
            logit (B,C): Variable that stores the logits for the next items in the session-parallel mini-batch
            hidden: GRU hidden state
        '''

        if self.embedding_dim == -1:
            embedded = self.onehot_encode(input)
            if self.training and self.dropout_input > 0: embedded = self.embedding_dropout(embedded)
            embedded = embedded.unsqueeze(0)
        else:
            embedded = input.unsqueeze(0)
            embedded = self.look_up(embedded)

        output, hidden = self.gru(embedded, hidden) #(num_layer, B, H)
        output = output.view(-1, output.size(-1))  #(B,H)
        logit = self.final_activation(self.h2o(output))

        return logit, hidden

    def init_emb(self):
        '''
        Initialize the one_hot embedding buffer, which will be used for producing the one-hot embeddings efficiently
        '''
        onehot_buffer = torch.FloatTensor(self.batch_size, self.output_size)
        onehot_buffer = onehot_buffer.to(self.device)
        return onehot_buffer

    def onehot_encode(self, input):
        """
        Returns a one-hot vector corresponding to the input
        Args:
            input (B,): torch.LongTensor of item indices
            buffer (B,output_size): buffer that stores the one-hot vector
        Returns:
            one_hot (B,C): torch.FloatTensor of one-hot vectors
        """
        self.onehot_buffer.zero_()
        index = input.view(-1, 1)
        one_hot = self.onehot_buffer.scatter_(1, index, 1)
        return one_hot

    def embedding_dropout(self, input):
        p_drop = torch.Tensor(input.size(0), 1).fill_(1 - self.dropout_input)
        mask = torch.bernoulli(p_drop).expand_as(input) / (1 - self.dropout_input)
        mask = mask.to(self.device)
        input = input * mask
        return input

    def init_hidden(self):
        '''
        Initialize the hidden state of the GRU
        '''
        try:
            h0 = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(self.device)
        except:
            self.device = 'cpu'
            h0 = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(self.device)
        return h0

In [128]:
input_size = len(train_data[105].items)
hidden_size = 100
num_layers = 1
output_size = input_size
batch_size = 64
dropout_input = 0
dropout_hidden = 0.5
embedding_dim = -1
final_act = 'tanh'
loss_type = 'TOP1-max'
optimizer_type = 'Adagrad'
lr = 0.05
weight_decay = 0
momentum = 0
eps = 1e-6
n_epochs = 5
time_sort = False
sigma = None

In [129]:
input_size

71

In [130]:
cuda = False

In [131]:
#use random seed defined
np.random.seed(22)
torch.manual_seed(22)

In [132]:
seed = 0
if cuda:
    torch.cuda.manual_seed(seed)

In [133]:
loss_function = LossFunction(loss_type=loss_type, use_cuda=cuda)

In [134]:
model = GRU4REC(input_size, hidden_size, output_size, final_act=final_act,
                            num_layers=num_layers, use_cuda=cuda, batch_size=batch_size,
                            dropout_input=dropout_input, dropout_hidden=dropout_hidden, embedding_dim=embedding_dim)

In [135]:
def init_model(model):
    global sigma
    if sigma is not None:
        for p in model.parameters():
            if sigma != -1 and sigma != -2:
                sigma = sigma
                p.data.uniform_(-sigma, sigma)
            elif len(list(p.size())) > 1:
                sigma = np.sqrt(6.0 / (p.size(0) + p.size(1)))
                if sigma == -1:
                    p.data.uniform_(-sigma, sigma)
                else:
                    p.data.uniform_(0, sigma)

In [136]:
init_model(model)

In [137]:
class Optimizer:
    def __init__(self, params, optimizer_type='Adagrad', lr=.05,
                 momentum=0, weight_decay=0, eps=1e-6):
        '''
        An abstract optimizer class for handling various kinds of optimizers.
        You can specify the optimizer type and related parameters as you want.
        Usage is exactly the same as an instance of torch.optim

        Args:
            params: torch.nn.Parameter. The NN parameters to optimize
            optimizer_type: type of the optimizer to use
            lr: learning rate
            momentum: momentum, if needed
            weight_decay: weight decay, if needed. Equivalent to L2 regulariztion.
            eps: eps parameter, if needed.
        '''
        if optimizer_type == 'RMSProp':
            self.optimizer = optim.RMSprop(params, lr=lr, eps=eps, weight_decay=weight_decay, momentum=momentum)
        elif optimizer_type == 'Adagrad':
            self.optimizer = optim.Adagrad(params, lr=lr, weight_decay=weight_decay)
        elif optimizer_type == 'Adadelta':
            self.optimizer = optim.Adadelta(params, lr=lr, eps=eps, weight_decay=weight_decay)
        elif optimizer_type == 'Adam':
            self.optimizer = optim.Adam(params, lr=lr, eps=eps, weight_decay=weight_decay)
        elif optimizer_type == 'SparseAdam':
            self.optimizer = optim.SparseAdam(params, lr=lr, eps=eps)
        elif optimizer_type == 'SGD':
            self.optimizer = optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
        else:
            raise NotImplementedError

    def zero_grad(self):
        self.optimizer.zero_grad()

    def step(self):
        self.optimizer.step()

In [138]:
optimizer = Optimizer(model.parameters(), optimizer_type=optimizer_type, lr=lr, weight_decay=weight_decay, momentum=momentum, eps=eps)

In [139]:
def get_recall(indices, targets): #recall --> wether next item in session is within top K=20 recommended items or not
    """
    Calculates the recall score for the given predictions and targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
    """
    targets = targets.view(-1, 1).expand_as(indices)
    hits = (targets == indices).nonzero()
    if len(hits) == 0:
        return 0
    n_hits = (targets == indices).nonzero()[:, :-1].size(0)
    recall = float(n_hits) / targets.size(0)
    return recall

In [140]:
def get_mrr(indices, targets): #Mean Receiprocal Rank --> Average of rank of next item in the session.
    """
    Calculates the MRR score for the given predictions and targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        mrr (float): the mrr score
    """
    tmp = targets.view(-1, 1)
    targets = tmp.expand_as(indices)
    hits = (targets == indices).nonzero()
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    rranks = torch.reciprocal(ranks)
    mrr = torch.sum(rranks).data / targets.size(0)
    return mrr

In [141]:
def evaluate(indices, targets, k=20):
    """
    Evaluates the model using Recall@K, MRR@K scores.

    Args:
        logits (B,C): torch.LongTensor. The predicted logit for the next items.
        targets (B): torch.LongTensor. actual target indices.

    Returns:
        recall (float): the recall score
        mrr (float): the mrr score
    """
    _, indices = torch.topk(indices, k, -1)
    recall = get_recall(indices, targets)
    mrr = get_mrr(indices, targets)
    return recall, mrr

In [142]:
class Evaluation(object):
    def __init__(self, model, loss_func, use_cuda, k=5):
        self.model = model
        self.loss_func = loss_func
        self.topk = k
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        #self.device = torch.device('cpu')

    def eval(self, eval_data, batch_size):
        self.model.eval()
        losses = []
        recalls = []
        mrrs = []
        dataloader = DataLoader(eval_data, batch_size)
        with torch.no_grad():
            hidden = self.model.init_hidden()
            for ii, (input, target, mask) in enumerate(dataloader):
            #for input, target, mask in dataloader:
                input = input.to(self.device)
                target = target.to(self.device)
                logit, hidden = self.model(input, hidden)
                logit_sampled = logit[:, target.view(-1)]
                loss = self.loss_func(logit_sampled)
                recall, mrr = evaluate(logit, target, k=self.topk)

                # torch.Tensor.item() to get a Python number from a tensor containing a single value
                losses.append(loss.item())
                recalls.append(recall)
                mrrs.append(mrr.cpu())
        mean_losses = np.mean(losses)
        mean_recall = np.mean(recalls)
        mean_mrr = np.mean(mrrs)
        #mean_mrr = 0

        return mean_losses, mean_recall, mean_mrr

In [143]:
class Trainer(object):
    def __init__(self, model, train_data, eval_data, optim, use_cuda, loss_func, batch_size):
        self.model = model
        self.train_data = train_data
        self.eval_data = eval_data
        self.optim = optim
        self.loss_func = loss_func
        self.evaluation = Evaluation(self.model, self.loss_func, use_cuda, k = 5)
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        #self.device = torch.device('cpu')
        self.batch_size = batch_size
        #self.args = args

    def train(self, start_epoch, end_epoch, start_time=None):
        if start_time is None:
            self.start_time = time.time()
        else:
            self.start_time = start_time

        for epoch in range(start_epoch, end_epoch + 1):
            st = time.time()
            print('Start Epoch #', epoch)
            train_loss = self.train_epoch(epoch)
            loss, recall, mrr = self.evaluation.eval(self.eval_data, self.batch_size)


            print("Epoch: {}, train loss: {:.4f}, loss: {:.4f}, recall: {:.4f}, mrr: {:.4f}, time: {}".format(epoch, train_loss, loss, recall, mrr, time.time() - st))
            checkpoint = {
                'model': self.model,
                'epoch': epoch,
                'optim': self.optim,
                'loss': loss,
                'recall': recall,
                'mrr': mrr
            }
            #model_name = os.path.join('checkpoint', "model_{0:05d}.pt".format(epoch))
            #torch.save(checkpoint, model_name)
            #print("Save model as %s" % model_name)


    def train_epoch(self, epoch):
        self.model.train()
        losses = []

        def reset_hidden(hidden, mask):
            """Helper function that resets hidden state when some sessions terminate"""
            if len(mask) != 0:
                hidden[:, mask, :] = 0
            return hidden

        hidden = self.model.init_hidden()
        dataloader = DataLoader(self.train_data, self.batch_size)
        #for ii,(data,label) in tqdm(enumerate(train_dataloader),total=len(train_data)):
        for ii, (input, target, mask) in enumerate(dataloader):
            input = input.to(self.device)
            target = target.to(self.device)
            self.optim.zero_grad()
            hidden = reset_hidden(hidden, mask).detach()
            logit, hidden = self.model(input, hidden)
            # output sampling
            logit_sampled = logit[:, target.view(-1)]
            loss = self.loss_func(logit_sampled)
            losses.append(loss.item())
            loss.backward()
            self.optim.step()

        mean_losses = np.mean(losses)
        return mean_losses

In [144]:
trainer = Trainer(model, train_data=train_data[105], eval_data=valid_data[105], optim=optimizer, use_cuda=cuda, loss_func=loss_function, batch_size=batch_size)

In [145]:
print('#### START TRAINING....')
trainer.train(0, n_epochs - 1)

#### START TRAINING....
Start Epoch # 0
Epoch: 0, train loss: 0.0144, loss: 0.0148, recall: 0.5547, mrr: 0.5068, time: 0.07386970520019531
Start Epoch # 1
Epoch: 1, train loss: 0.0140, loss: 0.0148, recall: 0.5547, mrr: 0.5013, time: 0.06340599060058594
Start Epoch # 2
Epoch: 2, train loss: 0.0139, loss: 0.0148, recall: 0.5547, mrr: 0.4928, time: 0.05922079086303711
Start Epoch # 3
Epoch: 3, train loss: 0.0139, loss: 0.0148, recall: 0.5469, mrr: 0.4918, time: 0.06041860580444336
Start Epoch # 4
Epoch: 4, train loss: 0.0139, loss: 0.0148, recall: 0.5469, mrr: 0.4910, time: 0.05916309356689453


## FL

In [40]:
#use random seed defined
np.random.seed(22)
torch.manual_seed(22)

In [41]:
class Trainer(object):
    def __init__(self, model, train_data, eval_data, optim, use_cuda, loss_func, batch_size, clientID = 0):
        self.model = model
        self.train_data = train_data
        self.eval_data = eval_data
        self.optim = optim
        self.loss_func = loss_func
        self.evaluation = Evaluation(self.model, self.loss_func, use_cuda, k = 5)
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        #self.device = torch.device('cpu')
        self.batch_size = batch_size
        self.clientID = clientID
        #self.args = args

    def train(self, start_epoch, end_epoch, start_time=None):
        if start_time is None:
            self.start_time = time.time()
        else:
            self.start_time = start_time

        for epoch in range(start_epoch, end_epoch + 1):
            st = time.time()
            print('Start Epoch #', self.clientID)
            train_loss = self.train_epoch(epoch)
            loss, recall, mrr = self.evaluation.eval(self.eval_data, self.batch_size)


            print("client: {}, train loss: {:.4f}, loss: {:.4f}, recall: {:.4f}, mrr: {:.4f}, time: {}".format(self.clientID, train_loss, loss, recall, mrr, time.time() - st))
            checkpoint = {
                'model': self.model,
                'epoch': epoch,
                'optim': self.optim,
                'loss': loss,
                'recall': recall,
                'mrr': mrr
            }
            #model_name = os.path.join('checkpoint', "model_{0:05d}.pt".format(epoch))
            #torch.save(checkpoint, model_name)
            #print("Save model as %s" % model_name)


    def train_epoch(self, epoch):
        self.model.train()
        losses = []

        def reset_hidden(hidden, mask):
            """Helper function that resets hidden state when some sessions terminate"""
            if len(mask) != 0:
                hidden[:, mask, :] = 0
            return hidden

        hidden = self.model.init_hidden()
        dataloader = DataLoader(self.train_data, self.batch_size)
        #for ii,(data,label) in tqdm(enumerate(train_dataloader),total=len(train_data)):
        for ii, (input, target, mask) in enumerate(dataloader):
            input = input.to(self.device)
            target = target.to(self.device)
            self.optim.zero_grad()
            hidden = reset_hidden(hidden, mask).detach()
            logit, hidden = self.model(input, hidden)
            # output sampling
            logit_sampled = logit[:, target.view(-1)]
            loss = self.loss_func(logit_sampled)
            losses.append(loss.item())
            loss.backward()
            self.optim.step()

        mean_losses = np.mean(losses)
        return mean_losses

In [42]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [43]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, clientID = 0):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.clientID = clientID

    def get_parameters(self):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        trainer = Trainer(model, train_data=self.trainloader, eval_data=self.valloader, optim=optimizer, use_cuda=False, loss_func=loss_function, batch_size=batch_size, clientID=self.clientID)
        trainer.train(0, 0)
        #print(len(self.trainloader.df))
        return get_parameters(self.net), len(self.trainloader.df), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        evaluation = Evaluation(self.net, loss_function, use_cuda= False, k = 5)
        loss, recall, mrr = evaluation.eval(self.valloader, 64)
        #loss, accuracy = test(self.net, self.valloader)
        #print ("print: ", recall)
        return float(loss), len(self.valloader.df), {"accuracy": float(recall)}



In [44]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    model = GRU4REC(87, hidden_size, 87, final_act=final_act,
                            num_layers=num_layers, use_cuda=False, batch_size=batch_size,
                            dropout_input=dropout_input, dropout_hidden=dropout_hidden, embedding_dim=embedding_dim).to(DEVICE)
    
    init_model(model)


    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = train_data[int(cid)]
    valloader = valid_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, trainloader, valloader, int(cid))



In [45]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_eval=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=1,  # Never sample less than 10 clients for training
        min_eval_clients=1,  # Never sample less than 5 clients for evaluation
        min_available_clients=1,  # Wait until all 10 clients are available
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS+1,
    num_rounds=5,
    strategy=strategy,
)

INFO flower 2022-06-17 12:42:50,284 | app.py:147 | Ray initialized with resources: {'object_store_memory': 4760811110.0, 'memory': 9521622222.0, 'GPU': 1.0, 'node:220.67.127.72': 1.0, 'accelerator_type:G': 1.0, 'CPU': 16.0}
INFO flower 2022-06-17 12:42:50,285 | app.py:156 | Starting Flower simulation running: {'num_rounds': 5}
INFO flower 2022-06-17 12:42:50,286 | server.py:128 | Initializing global parameters
INFO flower 2022-06-17 12:42:50,286 | server.py:327 | Requesting initial parameters from one random client
INFO flower 2022-06-17 12:42:51,131 | server.py:330 | Received initial parameters from one random client
INFO flower 2022-06-17 12:42:51,132 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-17 12:42:51,132 | server.py:143 | FL starting
DEBUG flower 2022-06-17 12:42:51,133 | server.py:269 | fit_round: strategy sampled 300 clients (out of 300)
(launch_and_get_parameters pid=318485) /home/tori/anaconda3/envs/flower3/lib/python3.7/site-packages/torch/nn/module

(launch_and_fit pid=318485) Start Epoch # 297
(launch_and_fit pid=318485) client: 297, train loss: 0.0144, loss: 0.0144, recall: 0.6484, mrr: 0.6217, time: 4.1579508781433105
(launch_and_fit pid=318482) Start Epoch # 104
(launch_and_fit pid=318484) Start Epoch # 273
(launch_and_fit pid=318491) Start Epoch # 113
(launch_and_fit pid=318486) Start Epoch # 102
(launch_and_fit pid=318483) Start Epoch # 272
(launch_and_fit pid=318496) Start Epoch # 24
(launch_and_fit pid=318494) Start Epoch # 285
(launch_and_fit pid=318489) Start Epoch # 85
(launch_and_fit pid=318493) Start Epoch # 230
(launch_and_fit pid=318495) Start Epoch # 145
(launch_and_fit pid=318490) Start Epoch # 97
(launch_and_fit pid=318497) Start Epoch # 259
(launch_and_fit pid=318488) Start Epoch # 41
(launch_and_fit pid=318487) Start Epoch # 112
(launch_and_fit pid=318492) Start Epoch # 111
(launch_and_fit pid=318491) client: 113, train loss: 0.0142, loss: 0.0147, recall: 0.6406, mrr: 0.6077, time: 0.15400290489196777
(launch_a

(launch_and_fit pid=318491) /home/tori/anaconda3/envs/flower3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
(launch_and_fit pid=318491)   "num_layers={}".format(dropout, num_layers))
(launch_and_fit pid=318496) /home/tori/anaconda3/envs/flower3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
(launch_and_fit pid=318496)   "num_layers={}".format(dropout, num_layers))
(launch_and_fit pid=318484) /home/tori/anaconda3/envs/flower3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and nu

(launch_and_fit pid=318484) client: 273, train loss: 0.0143, loss: 0.0146, recall: 0.5938, mrr: 0.5653, time: 0.20922112464904785
(launch_and_fit pid=318486) client: 102, train loss: 0.0144, loss: 0.0145, recall: 0.6615, mrr: 0.6044, time: 0.22640585899353027
(launch_and_fit pid=318483) client: 272, train loss: 0.0143, loss: 0.0143, recall: 0.6667, mrr: 0.6201, time: 0.19410419464111328
(launch_and_fit pid=318496) client: 24, train loss: 0.0142, loss: 0.0145, recall: 0.6953, mrr: 0.5940, time: 0.1509106159210205
(launch_and_fit pid=318494) client: 285, train loss: 0.0144, loss: 0.0144, recall: 0.6406, mrr: 0.6250, time: 0.17308998107910156
(launch_and_fit pid=318489) client: 85, train loss: 0.0143, loss: 0.0143, recall: 0.7188, mrr: 0.7047, time: 0.15874481201171875
(launch_and_fit pid=318493) client: 230, train loss: 0.0143, loss: 0.0146, recall: 0.5938, mrr: 0.5440, time: 0.16537714004516602
(launch_and_fit pid=318495) client: 145, train loss: 0.0143, loss: 0.0141, recall: 0.7188, mr

(raylet) Spilled 2096 MiB, 31 objects, write throughput 24 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

(launch_and_fit pid=318497) Start Epoch # 279
(launch_and_fit pid=318497) client: 279, train loss: 0.0141, loss: 0.0146, recall: 0.7422, mrr: 0.6980, time: 0.05710339546203613
(launch_and_fit pid=318497) Start Epoch # 1
(launch_and_fit pid=318485) Start Epoch # 35
(launch_and_fit pid=318485) client: 35, train loss: 0.0143, loss: 0.0143, recall: 0.6823, mrr: 0.6280, time: 0.06157565116882324
(launch_and_fit pid=318495) Start Epoch # 102
(launch_and_fit pid=318497) client: 1, train loss: 0.0143, loss: 0.0143, recall: 0.6797, mrr: 0.5926, time: 0.06122112274169922
(launch_and_fit pid=318495) client: 102, train loss: 0.0144, loss: 0.0145, recall: 0.6615, mrr: 0.6044, time: 0.0612177848815918
(launch_and_fit pid=318495) Start Epoch # 172
(launch_and_fit pid=318495) client: 172, train loss: 0.0143, loss: 0.0144, recall: 0.6719, mrr: 0.6219, time: 0.051888227462768555
(launch_and_fit pid=318495) Start Epoch # 222
(launch_and_fit pid=318495) client: 222, train loss: 0.0143, loss: 0.0144, recal

DEBUG flower 2022-06-17 13:46:02,125 | server.py:281 | fit_round received 297 results and 3 failures


(launch_and_fit pid=318495) Start Epoch # 147
(launch_and_fit pid=318495) client: 147, train loss: 0.0143, loss: 0.0143, recall: 0.6562, mrr: 0.6409, time: 0.0546727180480957


DEBUG flower 2022-06-17 13:46:03,365 | server.py:215 | evaluate_round: strategy sampled 150 clients (out of 300)
DEBUG flower 2022-06-17 13:48:02,891 | server.py:227 | evaluate_round received 149 results and 1 failures
INFO flower 2022-06-17 13:48:02,892 | server.py:182 | FL finished in 3911.759077187002
INFO flower 2022-06-17 13:48:02,894 | app.py:149 | app_fit: losses_distributed [(1, 0.01565447713119567), (2, 0.015653786061830377), (3, 0.01565523891953344), (4, 0.015654583338309897), (5, 0.015654574699045052)]
INFO flower 2022-06-17 13:48:02,894 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-17 13:48:02,895 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-06-17 13:48:02,895 | app.py:152 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.01565447713119567
	round 2: 0.015653786061830377
	round 3: 0.01565523891953344
	round 4: 0.015654583338309897
	round 5: 0.015654574699045052

In [147]:
import os
os.system('jupyter nbconvert --to html gru-FL-CategoryRecommendation-300.ipynb')

[NbConvertApp] Converting notebook gru-FL-CategoryRecommendation-300.ipynb to html
[NbConvertApp] Writing 1123120 bytes to gru-FL-CategoryRecommendation-300.html


0